In [0]:
from pyspark.sql.functions import first, rand, count
dates = spark.sql("SELECT explode(sequence(DATE'2024-01-01', DATE'2024-03-24', INTERVAL 1 DAY)) as  calendar_date")
c_id = spark.sql("SELECT explode(sequence(1,200, 1)) as  client_id")
types = spark.sql("""SELECT concat("col_", colName) as col_name from (SELECT explode(sequence(1,20, 1)) as  colName)""")
 
dates = dates.repartition(99)
c_id = c_id.repartition(11)
types = types.repartition(1)
 
df_cartesian = c_id.crossJoin(dates.select("calendar_date")).crossJoin(types.select("col_name")).select("client_id","calendar_date","col_name")
df_cartesian2 = df_cartesian.groupBy("calendar_date").agg(count("client_id"))
 
# display(df_cartesian2.limit(1000))
 
df_cartesian = df_cartesian.withColumn("val", (rand()*10).cast("int"))

df_grp = df_cartesian.groupBy("client_id","calendar_date").pivot("col_name").agg((first("val").alias("val")))

display(df_grp.head(10))


client_id,calendar_date,col_1,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_2,col_20,col_3,col_4,col_5,col_6,col_7,col_8,col_9
170,2024-03-19,2,9,6,0,9,3,2,9,7,6,2,1,7,5,7,4,8,2,8,0
170,2024-03-07,6,1,2,9,4,1,4,7,7,2,1,6,7,1,0,0,1,2,0,9
170,2024-01-17,6,1,6,5,4,9,7,9,5,8,3,9,1,6,5,2,4,4,8,6
170,2024-01-31,5,2,6,2,5,6,0,9,0,5,0,9,1,8,2,6,6,1,0,6
170,2024-02-26,9,9,8,5,1,2,9,0,1,1,1,4,5,0,9,0,7,3,5,4
170,2024-03-21,5,8,1,8,2,2,1,3,5,1,5,8,4,3,7,4,2,8,9,1
170,2024-01-29,8,7,4,7,5,2,8,1,0,7,8,7,7,7,8,3,3,1,1,4
170,2024-03-24,7,2,7,9,9,9,0,3,8,0,4,3,5,8,5,3,5,3,1,2
170,2024-02-24,9,4,9,0,7,8,7,2,8,6,0,2,5,2,2,0,6,0,9,1
170,2024-02-29,0,2,3,3,4,4,6,7,5,8,8,2,6,8,4,4,5,8,1,7


In [0]:
left_join=df_cartesian.join(df_cartesian2, df_cartesian.calendar_date==df_cartesian2.calendar_date, 'inner').drop(df_cartesian2.calendar_date).withColumnRenamed("count(client_id)", "total_clients")
display(left_join.head(10))

client_id,col_name,val,calendar_date,total_clients
170,col_1,2,2024-03-19,4000
170,col_2,6,2024-03-19,4000
170,col_3,6,2024-03-19,4000
170,col_4,5,2024-03-19,4000
170,col_5,9,2024-03-19,4000
170,col_6,5,2024-03-19,4000
170,col_7,8,2024-03-19,4000
170,col_8,7,2024-03-19,4000
170,col_9,9,2024-03-19,4000
170,col_10,0,2024-03-19,4000


In [0]:
left_join=df_cartesian.join(df_cartesian2, df_cartesian.calendar_date==df_cartesian2.calendar_date, 'left').drop(df_cartesian2.calendar_date).withColumnRenamed("count(client_id)", "total_clients")
display(left_join.head(10))

client_id,col_name,val,calendar_date,total_clients
170,col_1,2,2024-03-19,4000
170,col_2,6,2024-03-19,4000
170,col_3,6,2024-03-19,4000
170,col_4,5,2024-03-19,4000
170,col_5,9,2024-03-19,4000
170,col_6,5,2024-03-19,4000
170,col_7,8,2024-03-19,4000
170,col_8,7,2024-03-19,4000
170,col_9,9,2024-03-19,4000
170,col_10,0,2024-03-19,4000
